## Step 1: Loopnest Scheduling

Run Timeloop-Topk using either the baseline model or the effective bandwidth model (considering the cryptographic engine)

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import yaml
import shutil
from pathlib import Path

from utils import generate_arch_files, xml2mapping 

### Define the architecture

First, define an architecture design. The code below generates/detects a new architecture configuration based on the template design at `designs/{design_name}/template`.

In [ ]:
configuration_dict = {}

# template design (with constraints and memory hierarchy representing "dataflow")
configuration_dict['TEMPLATE_DESIGN'] = 'eyeriss_like'

# number of bits used for I/O/W; we assume integer
configuration_dict['WORDBITS'] = 16

# DRAM bandwidth setting: words / cycle (not bits / cycle)
configuration_dict['DRAM_READ_BANDWIDTH'] = 32
configuration_dict['DRAM_WRITE_BANDWIDTH'] = 32

# SRAM setting
# - do we have a single shared glb or multiple glbs for each datatype? 
# - for each glb (if shared, just one), define depth/width/#banks and bandwidths
configuration_dict['SRAM_SHARED'] = True
configuration_dict['SRAM_DEPTH'] = [2 ** 13]
configuration_dict['SRAM_WIDTH'] = [2 ** 7]
configuration_dict['SRAM_BANKS'] = [32]                     # SRAM width and SRAM banks define the maximum possible bandwidth
configuration_dict['SRAM_READ_BANDWIDTH'] = [32]
configuration_dict['SRAM_WRITE_BANDWIDTH'] = [32]

# PE array setting
# - shape of PE array X x Y
# - whether a PE has a shared scratchpad or separate scratchpads for each datatype
configuration_dict['PE_X'] = 14
configuration_dict['PE_Y'] = 12
configuration_dict['PE_SPAD_SHARED'] = False
configuration_dict['PE_SPAD_DEPTH'] = [192, 12, 16]         # Weight, IFmap, OFmap
configuration_dict['PE_SPAD_WIDTH'] = [16, 16, 16]

# Cryptographic engine setting
# - type of cryptographic engine + dram (LPDDR4 + AES-GCM)
# - cycle for AES-GCM 
# - whether the cryptographic engines are shared among all datatypes or assigned to each datatype
configuration_dict['CRYPT_ENGINE_TYPE'] = 'effective_lpddr4_aesgcm'
configuration_dict['CRYPT_ENGINE_CYCLE_PER_BLOCK'] = 11            # avg. cycle/128bit

configuration_dict['CRYPT_ENGINE_SHARED'] = False
configuration_dict['CRYPT_ENGINE_COUNT'] = [1, 1, 1]

configuration_dict['EFFECTIVE_CONSERVATIVE'] = True

# Create directory for this configuration if it doesn't exist already
# iterate through design folders to check if any pre-exisiting folder
design_dir = 'designs/{}'.format(configuration_dict['TEMPLATE_DESIGN'])
arch_dir = None
total_vers = 0
for path in os.listdir(design_dir):
    if path != 'template' and os.path.isdir(os.path.join(design_dir, path)):
        try:
            with open(os.path.join(design_dir, path, 'config.yaml'), 'r') as f:
                config_file = yaml.safe_load(f)
            total_vers += 1
            if config_file == configuration_dict:
                arch_dir = path
                print("Pre-existing folder found. Setting the arch_dir to {}".format(arch_dir))
                break
        except:
            print("No config.yaml file in the directory {}".format(str(os.path.join(design_dir, path))))
            
if arch_dir is None:
    arch_dir = 'ver{}'.format(total_vers)
    shutil.copytree(os.path.join(design_dir, 'template'), os.path.join(design_dir, arch_dir))
    with open(os.path.join(design_dir, arch_dir, 'config.yaml'), 'w') as f:
        _ = yaml.dump(configuration_dict, f)
    
    # create baseline and effective files
    generate_arch_files(os.path.join(design_dir, arch_dir, 'arch'), configuration_dict)
    
    # create scheduling / evaluation folder
    os.mkdir(os.path.join(design_dir, arch_dir, 'scheduling'))
    os.mkdir(os.path.join(design_dir, arch_dir, 'evaluation'))
    
    # create folders for baseline scheduling / evaluation
    os.mkdir(os.path.join(design_dir, arch_dir, 'baseline_scheduling'))
    os.mkdir(os.path.join(design_dir, arch_dir, 'baseline_evaluation'))

..else if you know which folder you want to use, specify here instead of running the above cell

In [ ]:
design_dir = 'designs/{}'.format('eyeriss_like') # define your design name here

arch_ver = 0
arch_dir = 'ver{}'.format(arch_ver)              # sub directory under designs/{name}/{arch_dir}
with open(os.path.join(design_dir, arch_dir, 'config.yaml'), 'r') as f:
    configuration_dict = yaml.safe_load(f)
print("Setting the architecture directory to: {}".format(os.path.join(design_dir, arch_dir)))
print("Printing configuration:")
for key, value in configuration_dict.items():
    print("{}: {}".format(key, value))

### Define the DNN workload

Define a DNN workload in PyTorch, and convert it into a Timeloop workload.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision.models as model_zoo

import pytorch2timeloop as pytorch2timeloop

# Note: this version only supports nn.Conv2d (both normal convs and depthwise/pointwise convs) and nn.Linear

# AlexNet
model_name = 'alexnet'
net = model_zoo.alexnet(pretrained=False)

# ResNet18
# model_name = 'resnet18'
# net = model_zoo.resnet18(pretrained=False)

# MobilenetV2
# model_name = 'mobilenet_v2'
# net = model_zoo.mobilenet_v2(pretrained=False)

# Input / Batch info
input_size = (3, 224, 224)
batch_size = 1

print(net)

# Convert to timeloop workloads; stored in workloads/{model_name}_batch{batch_size}
top_dir = 'workloads'
sub_dir = '{}_batch{}'.format(model_name, batch_size)
exception_module_names = []

overwrite = False
if not os.path.exists(os.path.join(top_dir, sub_dir)) or overwrite:
    pytorch2timeloop.convert_model(
            net,
            input_size,
            batch_size,
            sub_dir,
            top_dir,
            True,
            exception_module_names
        )

In [ ]:
# Check duplicate layers (layer information is identical)
# For a per-layer loopnest scheduling, only unique layers have to be searched

base_dir = Path(os.getcwd())
timeloop_dir = 'designs/{}/{}'.format(configuration_dict['TEMPLATE_DESIGN'], arch_dir)

n_layers = 0
layer_dict = {}
layer_duplicate_info = {}
unique_layers = []
for module in net.modules():
    if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
        n_layers += 1
        if n_layers not in layer_dict.keys():
            workload_path = os.path.join(base_dir, top_dir, sub_dir, '{}_layer{}.yaml'.format(sub_dir, n_layers))
            with open(workload_path, 'r') as f:
                workload_info = yaml.safe_load(f)
            layer_dict[n_layers] = workload_info
        
        # identify the earliest duplicate layer
        for key in range(1, n_layers):
            if layer_dict[key] == layer_dict[n_layers]:
                layer_duplicate_info[n_layers] = key
                break
        if n_layers not in layer_duplicate_info:
            unique_layers.append(n_layers)
            
print(layer_duplicate_info)
print(unique_layers)

### Define the top-k parameter for Timeloop

Prepare the mapper.yaml for top-k search

In [ ]:
topk = 6
mapper_file_path = os.path.join(base_dir, timeloop_dir, 'mapper/mapper.yaml')
with open(mapper_file_path, 'r') as f:
    mapper_config = yaml.safe_load(f)
mapper_config['mapper']['topk'] = topk
with open(mapper_file_path, 'w') as f:
    _ = yaml.dump(mapper_config, f)

### Run Timeloop for the *baseline* model (w/o considering cryptographic engines)

Run timeloop for each unique layer, and convert the output to a mapping file in yaml format. Then, evaluate using the top-1 loopnest schedule for each unique layer.

Running this cell can take some time depending on your model and timeloop setting..

In [ ]:
def get_cmd(workload_info, layer_id, base_dir, timeloop_dir, sub_dir, top_dir):
    cwd = f"{base_dir/timeloop_dir/'baseline_scheduling'/sub_dir/f'layer{layer_id}'}"
    if 'M' in workload_info['problem']['instance']:
        constraint_pth = base_dir/timeloop_dir/'constraints/*.yaml'
    else:
        # depthwise
        constraint_pth = base_dir/timeloop_dir/'constraints_dw/*.yaml'

    timeloopcmd = f"timeloop-mapper-topk " \
                  f"{base_dir/timeloop_dir/'arch/baseline.yaml'} " \
                  f"{base_dir/timeloop_dir/'arch/components/*.yaml'} " \
                  f"{base_dir/timeloop_dir/'mapper/mapper.yaml'} " \
                  f"{constraint_pth} " \
                  f"{base_dir/top_dir/sub_dir/sub_dir}_layer{layer_id}.yaml "
    return [cwd, timeloopcmd]

cwd_list = []
cmd_list = []

for layer_id in unique_layers:
    workload_path = os.path.join(base_dir, top_dir, sub_dir, '{}_layer{}.yaml'.format(sub_dir, layer_id))
    with open(workload_path, 'r') as f:
        workload_info = yaml.safe_load(f)
    [cwd, cmd] = get_cmd(workload_info, layer_id, base_dir, timeloop_dir, sub_dir, top_dir)
    cwd_list.append(cwd)
    cmd_list.append(cmd)
    
if not os.path.exists(os.path.join(base_dir, timeloop_dir, 'baseline_scheduling', sub_dir)):
    os.mkdir(os.path.join(base_dir, timeloop_dir, 'baseline_scheduling', sub_dir))
for cwd, cmd in zip(cwd_list, cmd_list):
    print("Executing cmd: {}".format(cmd))
    try:
        os.chdir(cwd)
    except:
        os.mkdir(cwd)
        os.chdir(cwd)
    os.system(cmd)
os.chdir(base_dir)

Convert to mapping (.yaml) files

In [ ]:
def convert_to_mapping(base_dir, timeloop_dir, top_dir, sub_dir, layer_idx, topk_idx):
    xml_file = os.path.join(base_dir, timeloop_dir, 'baseline_scheduling', sub_dir, "layer{}".format(layer_idx), \
                            "timeloop-mapper-topk{}.map+stats.xml".format(topk_idx))
    workload_file = os.path.join(base_dir, top_dir, sub_dir, "{}_layer{}.yaml".format(sub_dir, layer_idx))
    with open(workload_file, 'r') as f:
        workload_info = yaml.safe_load(f)
    if 'M' in workload_info['problem']['instance']:
        dw = False
    else:
        dw = True
    arch_constraint_file = os.path.join(base_dir, timeloop_dir, 'constraints_dw' if dw else 'constraints' , \
                                        'eyeriss_like_arch_constraints.yaml' if (configuration_dict['TEMPLATE_DESIGN'] == 'eyeriss_like' or \
                                                                                 configuration_dict['TEMPLATE_DESIGN'] == 'eyeriss_like_hbm2') \
                                        else 'simple_output_stationary_arch_constraints.yaml' if configuration_dict['TEMPLATE_DESIGN'] == 'output_stationary' \
                                        else 'simple_weight_stationary_arch_constraints.yaml')
    mapping = xml2mapping(xml_file, workload_file, arch_constraint_file, dw)
    with open(os.path.join(base_dir, timeloop_dir, 'baseline_scheduling',sub_dir, "layer{}".format(layer_idx), \
                           "mapping{}.yaml".format(topk_idx)), 'w') as f:
        _ = yaml.dump({'mapping': mapping}, f)
        
for layer_idx in unique_layers:
    for k in range(1, topk + 1):
        convert_to_mapping(base_dir, timeloop_dir, top_dir, sub_dir, layer_idx, k)

Evaluate the top-1 loopnest schedule

In [ ]:
def get_cmd_model(workload_info, layer_id, base_dir, timeloop_dir, sub_dir, top_dir):
    cwd = f"{base_dir/timeloop_dir/'baseline_evaluation'/sub_dir/f'layer{layer_id}'}"
    if 'M' in workload_info['problem']['instance']:
        constraint_pth = base_dir/timeloop_dir/'constraints/*.yaml'
    else:
        # depthwise
        constraint_pth = base_dir/timeloop_dir/'constraints_dw/*.yaml'

    timeloopcmd = f"timeloop-model " \
                  f"{base_dir/timeloop_dir/'arch/baseline.yaml'} " \
                  f"{base_dir/timeloop_dir/'arch/components/*.yaml'} " \
                  f"{base_dir/timeloop_dir/'baseline_scheduling'/sub_dir/f'layer{layer_id}/mapping1.yaml'} " \
                  f"{base_dir/top_dir/sub_dir/sub_dir}_layer{layer_id}.yaml "
    return [cwd, timeloopcmd]

cwd_list = []
cmd_list = []
for layer_id in unique_layers:
    workload_path = os.path.join(base_dir, top_dir, sub_dir, '{}_layer{}.yaml'.format(sub_dir, layer_id))
    with open(workload_path, 'r') as f:
        workload_info = yaml.safe_load(f)
    [cwd, cmd] = get_cmd_model(workload_info, layer_id, base_dir, timeloop_dir, sub_dir, top_dir)
    cwd_list.append(cwd)
    cmd_list.append(cmd)
    
if not os.path.exists(os.path.join(base_dir, timeloop_dir, 'baseline_evaluation', sub_dir)):
    os.mkdir(os.path.join(base_dir, timeloop_dir, 'baseline_evaluation', sub_dir))
for cwd, cmd in zip(cwd_list, cmd_list):
    print("Executing cmd: {}".format(cmd))
    try:
        os.chdir(cwd)
    except:
        os.mkdir(cwd)
        os.chdir(cwd)
    os.system(cmd)
os.chdir(base_dir)

### Run Timeloop for the *effective* model (considering cryptographic engines)

In [ ]:
import time

def get_cmd(workload_info, layer_id, base_dir, timeloop_dir, sub_dir, top_dir):
    cwd = f"{base_dir/timeloop_dir/'scheduling'/sub_dir/f'layer{layer_id}'}"
    if 'M' in workload_info['problem']['instance']:
        constraint_pth = base_dir/timeloop_dir/'constraints/*.yaml'
    else:
        # depthwise
        constraint_pth = base_dir/timeloop_dir/'constraints_dw/*.yaml'

    timeloopcmd = f"timeloop-mapper-topk " \
                  f"{base_dir/timeloop_dir/'arch/effective.yaml'} " \
                  f"{base_dir/timeloop_dir/'arch/components/*.yaml'} " \
                  f"{base_dir/timeloop_dir/'mapper/mapper.yaml'} " \
                  f"{constraint_pth} " \
                  f"{base_dir/top_dir/sub_dir/sub_dir}_layer{layer_id}.yaml "
    return [cwd, timeloopcmd]

cwd_list = []
cmd_list = []

for layer_id in unique_layers:
    workload_path = os.path.join(base_dir, top_dir, sub_dir, '{}_layer{}.yaml'.format(sub_dir, layer_id))
    with open(workload_path, 'r') as f:
        workload_info = yaml.safe_load(f)
    [cwd, cmd] = get_cmd(workload_info, layer_id, base_dir, timeloop_dir, sub_dir, top_dir)
    cwd_list.append(cwd)
    cmd_list.append(cmd)
    
if not os.path.exists(os.path.join(base_dir, timeloop_dir, 'scheduling', sub_dir)):
    os.mkdir(os.path.join(base_dir, timeloop_dir, 'scheduling', sub_dir))
    
start_time = time.time()
for cwd, cmd in zip(cwd_list, cmd_list):
    print("Executing cmd: {}".format(cmd))
    try:
        os.chdir(cwd)
    except:
        os.mkdir(cwd)
        os.chdir(cwd)
    os.system(cmd)
os.chdir(base_dir)

# Time this cell
print("Execution time: {}s".format(time.time() - start_time))

In [ ]:
def convert_to_mapping(base_dir, timeloop_dir, top_dir, sub_dir, layer_idx, topk_idx):
    xml_file = os.path.join(base_dir, timeloop_dir, 'scheduling', sub_dir, "layer{}".format(layer_idx), \
                            "timeloop-mapper-topk{}.map+stats.xml".format(topk_idx))
    workload_file = os.path.join(base_dir, top_dir, sub_dir, "{}_layer{}.yaml".format(sub_dir, layer_idx))
    # print(workload_file)
    with open(workload_file, 'r') as f:
        workload_info = yaml.safe_load(f)
    if 'M' in workload_info['problem']['instance']:
        dw = False
    else:
        dw = True
    arch_constraint_file = os.path.join(base_dir, timeloop_dir, 'constraints_dw' if dw else 'constraints' , \
                                        'eyeriss_like_arch_constraints.yaml' if (configuration_dict['TEMPLATE_DESIGN'] == 'eyeriss_like' \
                                                                                 or configuration_dict['TEMPLATE_DESIGN'] == 'eyeriss_like_hbm2') \
                                        else 'simple_output_stationary_arch_constraints.yaml' if configuration_dict['TEMPLATE_DESIGN'] == 'output_stationary' \
                                        else 'simple_weight_stationary_arch_constraints.yaml')
    # print(layer_idx, dw)
    mapping = xml2mapping(xml_file, workload_file, arch_constraint_file, dw)
    with open(os.path.join(base_dir, timeloop_dir, 'scheduling',sub_dir, "layer{}".format(layer_idx), \
                           "mapping{}.yaml".format(topk_idx)), 'w') as f:
        _ = yaml.dump({'mapping': mapping}, f)
        
for layer_idx in unique_layers:
    for k in range(1, topk + 1):
        convert_to_mapping(base_dir, timeloop_dir, top_dir, sub_dir, layer_idx, k)

In [ ]:
def get_cmd_model(workload_info, layer_id, base_dir, timeloop_dir, sub_dir, top_dir):
    cwd = f"{base_dir/timeloop_dir/'evaluation'/sub_dir/f'layer{layer_id}'}"
    if 'M' in workload_info['problem']['instance']:
        constraint_pth = base_dir/timeloop_dir/'constraints/*.yaml'
    else:
        # depthwise
        constraint_pth = base_dir/timeloop_dir/'constraints_dw/*.yaml'

    timeloopcmd = f"timeloop-model " \
                  f"{base_dir/timeloop_dir/'arch/baseline.yaml'} " \
                  f"{base_dir/timeloop_dir/'arch/components/*.yaml'} " \
                  f"{base_dir/timeloop_dir/'scheduling'/sub_dir/f'layer{layer_id}/mapping1.yaml'} " \
                  f"{base_dir/top_dir/sub_dir/sub_dir}_layer{layer_id}.yaml "
    return [cwd, timeloopcmd]

cwd_list = []
cmd_list = []
for layer_id in unique_layers:
    workload_path = os.path.join(base_dir, top_dir, sub_dir, '{}_layer{}.yaml'.format(sub_dir, layer_id))
    with open(workload_path, 'r') as f:
        workload_info = yaml.safe_load(f)
    [cwd, cmd] = get_cmd_model(workload_info, layer_id, base_dir, timeloop_dir, sub_dir, top_dir)
    cwd_list.append(cwd)
    cmd_list.append(cmd)
    
if not os.path.exists(os.path.join(base_dir, timeloop_dir, 'evaluation', sub_dir)):
    os.mkdir(os.path.join(base_dir, timeloop_dir, 'evaluation', sub_dir))
for cwd, cmd in zip(cwd_list, cmd_list):
    print("Executing cmd: {}".format(cmd))
    try:
        os.chdir(cwd)
    except:
        os.mkdir(cwd)
        os.chdir(cwd)
    os.system(cmd)
os.chdir(base_dir)